In [1]:
from tensorflow.python import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas
from tensorflow.keras import utils

/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 75, 75
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 14
# Количество изображений для обучения
nb_train_samples = 224
# Количество изображений для проверки
nb_validation_samples = 28
# Количество изображений для тестирования
nb_test_samples = 28

In [3]:
classes = ['круассан','пончик', 'завиток', 'полки']

In [4]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
vgg16_net.trainable = False
#vgg16_net.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
model = Sequential()
#model.add(Conv2D(32, (3, 3), input_shape=input_shape))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(32, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-6),
              metrics=['accuracy'])

In [7]:
datagen = ImageDataGenerator(rescale=1./255., 
                             rotation_range=45,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.2,
                             channel_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True)

In [16]:
test_df = pandas.read_csv("test.csv")
train_df = pandas.read_csv("train.csv")
val_df = pandas.read_csv("val.csv")
x_col="filename"
y_col="label"
y_col = utils.to_categorical(y_col, 4)
print(y_col)

train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=None,
        #x_col="filename",
        #y_col="label",
        subset="training",
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

val_generator = datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=None,
        #x_col="filename",
        #y_col="label",
        #y_col = utils.to_categorical(y_col, 4),
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=None,
        #x_col="filename",
        #y_col="label",
        #y_col = utils.to_categorical(y_col, 4),
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='categorical')


ValueError: invalid literal for int() with base 10: 'label'

In [ ]:
y_col = utils.to_categorical(y_col, 4)
print(y_col)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)


In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block3_conv1':
        trainable = True
    layer.trainable = trainable

In [ ]:
#model.compile(loss='categorical_crossentropy',
#              optimizer=Adam(lr=1e-5), 
#              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Сохраняем сеть на каждой эпохе
# {epoch:02d} - номер эпохи
# {val_acc:.4f} - значение доли верных значений на проверочном наборе данных
callbacks = [ModelCheckpoint('saveModelWeightStep/mnist-dense-{epoch:02d}-{val_acc:.4f}.hdf5')]
# Сохраняем только лучший вариант сети
#callbacks = [ModelCheckpoint('save/mnist-dense.hdf5', monitor='val_loss', save_best_only=True)]

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size, 
    callbacks=callbacks)

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
#model.save('75x75.h5')

In [ ]:
#print("Сохраняем сеть")
# Сохраняем сеть для последующего использования
# Генерируем описание модели в формате json
#model_json = model.to_json()
#json_file = open("mnist_model.json", "w")
# Записываем архитектуру сети в файл
#json_file.write(model_json)
#json_file.close()
# Записываем данные о весах в файл
#model.save_weights("mnist_model.h5")
#print("Сохранение сети завершено")

In [ ]:
#model = keras.models.load_model('75x75.h5')

In [ ]:
# создаем csv-файл со списком тестовых картинок и отображаем результаты теста по каждой
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas
spisoktest = pandas.read_csv("test.csv")
for k in range(len(spisoktest)):
    print(spisoktest.iloc[k,0])
    im = spisoktest.iloc[k,0]
    img = image.load_img(im, target_size=(img_width, img_height))
    # Преобразуем картинку в массив
    x = image.img_to_array(img)
    x = x.reshape(1, x.shape[0],x.shape[1],x.shape[2])
    # Нормализуем изображение
    x /= 255
    prediction = model.predict(x)
    prediction
    prediction = np.argmax(prediction)
    print("Номер класса:", prediction)
    print("Название класса:", classes[prediction])
        